<h1 align="center">Segmenting and Clustering Neighborhoods in Toronto</h1>

## Part 1 - To scrape the Toronto postal data from Wiki

### 1. Scrape the Wikipedia page

In [1]:
import requests
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plot
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
html_tree = BeautifulSoup(html_data, 'html5lib')

### 2. Parse the table data and create a DataFrame with PostalCode, Borough and Neighborhood columns

In [2]:
trs = html_tree.find('tbody').find_all('tr')
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column = ['PostalCode', 'Borough', 'Neighborhood']
toronto_df = pd.DataFrame(columns=column)
# extract raw data from html table to dataframe.
for tr in trs:
    tds = tr.find_all('td')
    if(tds != []):
        toronto_df = toronto_df.append({'PostalCode':tds[0].text.strip(), 'Borough':tds[1].text.strip(), 'Neighborhood':tds[2].text.strip()}, ignore_index=True)
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
# remove Borough value = 'Not assigned'
index_na = toronto_df[(toronto_df['Borough']=='Not assigned')].index
toronto_df.drop(index=index_na, inplace=True)
toronto_df.reset_index(inplace=True)
toronto_df.drop(columns='index', axis=1, inplace=True)
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
if(toronto_df[(toronto_df['Neighborhood']=='Not assigned')].shape[0]!=0):
    print('Found Neighborhood is not assigned; use Borough value')
    toronto_df.loc[toronto_df['Neighborhood']=='Not assigned', 'Neighborhood']=toronto_df.loc[toronto_df['Neighborhood']=='Not assigned', 'Borough'] 
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print(toronto_df.shape)
print('The dataframe has {} rows with {} columns'.format(toronto_df.shape[0], toronto_df.shape[1]))
    

(103, 3)
The dataframe has 103 rows with 3 columns
